In [ ]:
import os
import cv2
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models
from torch.utils.data import DataLoader, random_split

# Hyperparameters
IMAGE_SIZE = (64, 64)
BATCH_SIZE = 32
EPOCHS = 10

# Define transforms for the dataset (with augmentation)
data_transforms = transforms.Compose([
    transforms.ToPILImage(),  # Ensure compatibility with transforms
    transforms.Resize(IMAGE_SIZE),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

# Custom dataset loader to use OpenCV
def cv2_loader(path):
    image = cv2.imread(path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB
    return image

# Load dataset
dataset_path = "../datasets/dataset1"
dataset = datasets.ImageFolder(root=os.path.join(dataset_path), transform=data_transforms, loader=cv2_loader)

# Split dataset (85% train, 15% validation)
train_size = int(0.85 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4, pin_memory=True)

# Load pretrained ResNet18 model
model = models.resnet18(pretrained=True)
model.fc = nn.Sequential(
    nn.Linear(model.fc.in_features, 128),
    nn.ReLU(),  # Activation function changed to ReLU
    nn.Dropout(0.5),
    nn.Linear(128, 1),
    nn.Sigmoid()
)
model.to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))

# Define loss function and optimizer
criterion = nn.BCELoss()
optimizer = optim.AdamW(model.parameters(), lr=0.001)  # AdamW optimizer
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.5)  # Learning rate scheduler

# Training loop
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
scaler = torch.cuda.amp.GradScaler()

for epoch in range(EPOCHS):
    model.train()
    train_loss, train_correct = 0, 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.float().to(device)
        labels = labels.view(-1, 1)
        optimizer.zero_grad()
        
        with torch.cuda.amp.autocast():
            outputs = model(images)
            loss = criterion(outputs, labels)
        
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        
        train_loss += loss.item()
        train_correct += ((outputs > 0.5).float() == labels).sum().item()
    
    train_acc = train_correct / train_size
    scheduler.step()
    print(f"Epoch {epoch+1}/{EPOCHS}, Train Loss: {train_loss/len(train_loader):.4f}, Train Accuracy: {train_acc:.4f}")

# Validation
torch.no_grad()
model.eval()
val_correct = 0
with torch.no_grad():
    for images, labels in val_loader:
        images, labels = images.to(device), labels.float().to(device)
        labels = labels.view(-1, 1)
        outputs = model(images)
        val_correct += ((outputs > 0.5).float() == labels).sum().item()

val_acc = val_correct / val_size
print(f"Final Validation Accuracy: {val_acc:.4f}")
